In [328]:
#import packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import glob

#define path
path = r"/Users/valeriyamalenko/Desktop/trading/trading/momentum/data/mecklai-finance/"

#list paths to all files in directory
files=glob.glob(path +"/usd-*.csv")

#open the first file
df = pd.read_csv(files[0])

#append all other files in the directory to the first file
for i in files[1:]:
    df1=pd.read_csv(i, header=0)
    df=df.append(df1)

In [329]:
#set date to be the index
df=df.set_index('Date')

#convert date to datetime
df.index=pd.to_datetime(df.index)

#rename the column
df.rename(columns={'col1': 'fwd1m'}, inplace=True)

#get rid of missing data
df = df.loc[df["SpotRate"]!=0]
df = df.loc[df["fwd1m"]!=0]

#new cols for logs of spot and fwd rates
df['logSpotRate']=np.log(df.SpotRate)
df['logfwd1m']=np.log(df.fwd1m)

#select desired columns
df=df[['Currency','logSpotRate', 'logfwd1m']]

#first.groupby('Currency')['logSpotRate'].plot(legend=True)

# for each month, select last day of the month
df=df.loc[df.reset_index().groupby(df.index.to_period('M'))['Date'].max()]

#exclude data for 2019
dt = datetime.datetime(2019, 1, 1)
df=df[df.index < np.datetime64(dt)]


In [339]:
#checking for dupicates
ccy = df['Currency'].unique().tolist()
months=df.index.unique().tolist()

l =[]
for i in months:
    d=df.loc[df.index==i]
    b=len(d.Currency)
    l.append([i,b])
len(l)
l.sort()

In [310]:
ccy_list=df["Currency"].unique().tolist()
n_ccy=len(ccy_list)

#take a ccy data set
df1=df.loc[df['Currency']==ccy_list[0]]
#shift the fwd rate down
df1['shiftlogfwd1m']=df1.logfwd1m.shift(1)
#calculate return
df1['ret']=df1['shiftlogfwd1m']-df1['logSpotRate']

#repeat this for all ccies
for i in ccy_list[1:]:
    tb=df.loc[df['Currency']==i]
    tb['shiftlogfwd1m']=tb.logfwd1m.shift(1)
    tb['ret']=tb['shiftlogfwd1m']-tb['logSpotRate']
    #append all ccy dfs to form one df
    df1=df1.append(tb)    

/Users/valeriyamalenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/valeriyamalenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/valeriyamalenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,Currency,logSpotRate,logfwd1m,shiftlogfwd1m,ret
Date,,,,,
2011-01-31,USD/AED,1.301063,1.301145,NaN,NaN
2011-02-28,USD/AED,1.301036,1.301090,1.301145,0.000109
2011-03-31,USD/AED,1.301009,1.301090,1.301090,0.000082
2011-04-29,USD/AED,1.301172,1.301172,1.301090,-0.000082
2011-05-31,USD/AED,1.301036,1.301063,1.301172,0.000136
2011-06-30,USD/AED,1.301009,1.301036,1.301063,0.000054
2011-07-28,USD/AED,1.301036,1.301009,1.301036,0.000000
2011-08-25,USD/AED,1.301009,1.300982,1.301009,0.000000
2011-09-30,USD/AED,1.301199,1.301145,1.300982,-0.000218
